In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from gen import generation

excel_file = 'PID&SI.xlsx'
pd_data = pd.read_excel(excel_file, sheet_name='PDs')
suicide_data = pd.read_excel(excel_file, sheet_name='Suicide')

assert pd_data['serial'].equals(suicide_data['serial'])

X = pd_data.iloc[:, 1:26]
y = suicide_data['SI'] 

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Train set:", X_train.shape, y_train.shape)
print("Test set:", X_test.shape, y_test.shape)
print("Validation set:", X_val.shape, y_val.shape)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)

class SuicideRiskMLP(nn.Module):
    def __init__(self):
        super(SuicideRiskMLP, self).__init__()
        self.fc1 = nn.Linear(25, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 14)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

    
def train(model, optimizer, criterion, sample):
    model.train()
    total_loss = 0

    for i in range(sample):
        optimizer.zero_grad()
        input = X_train_tensor[i]
        label = y_train_tensor[i]
        params = model(input)
        pred_result = generation(params).squeeze(-1)
        loss = criterion(pred_result, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(total_loss)

Train set: (838, 25) (838,)
Test set: (280, 25) (280,)
Validation set: (280, 25) (280,)


In [6]:
model1 = SuicideRiskMLP()
criterion = nn.BCELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
num_epochs = 30
for epoch in range(num_epochs):
    model1.train()
    total_loss = 0

    for i in range(50):
        optimizer.zero_grad()
        input = X_train_tensor[i]
        label = y_train_tensor[i]
        params = model1(input)
        pred_result = generation(params).squeeze(-1)
        loss = criterion(pred_result, label)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    print("epoch: ", epoch, " loss: ", total_loss)

epoch:  0  loss:  91.73946711457393
epoch:  1  loss:  37.76447095256299
epoch:  2  loss:  34.77809787914157
epoch:  3  loss:  32.69761935714632
epoch:  4  loss:  32.70580733008683
epoch:  5  loss:  30.41028437949717
epoch:  6  loss:  31.07433419395238
epoch:  7  loss:  29.016595857217908
epoch:  8  loss:  29.59547765366733
epoch:  9  loss:  28.034149248618633
epoch:  10  loss:  30.812334273941815
epoch:  11  loss:  26.709225160069764
epoch:  12  loss:  27.421477164374664
epoch:  13  loss:  23.03218270186335
epoch:  14  loss:  21.595563870389014
epoch:  15  loss:  21.526283767423593
epoch:  16  loss:  20.69553782074945
epoch:  17  loss:  20.45206642683479
epoch:  18  loss:  18.129262479720637
epoch:  19  loss:  17.0380091286097
epoch:  20  loss:  18.42891854827394
epoch:  21  loss:  13.881704829633236
epoch:  22  loss:  10.297905685789374
epoch:  23  loss:  8.750003327304853
epoch:  24  loss:  18.540874932018596
epoch:  25  loss:  8.187627448522107
epoch:  26  loss:  7.66870599096228
ep

In [10]:
import copy

model2 = copy.deepcopy(model1)
criterion2 = nn.BCELoss()
optimizer2 = optim.Adam(model2.parameters(), lr=0.0001)
epochs = 100

for epoch in range(epochs):
    model2.train()
    total_loss = 0

    for i in range(780):
        optimizer2.zero_grad()
        input = X_train_tensor[i+50]
        label = y_train_tensor[i+50]
        params = model2(input)
        pred_result = generation(params).squeeze(-1)
        loss = criterion2(pred_result, label)
        loss.backward()
        optimizer2.step()
        
        total_loss += loss.item()

    print("epoch: ", epoch, " loss: ", total_loss)

epoch:  0  loss:  628.1432833584636
epoch:  1  loss:  487.95516322390176
epoch:  2  loss:  462.1147461095825
epoch:  3  loss:  452.64680490270257
epoch:  4  loss:  444.89214462786913
epoch:  5  loss:  436.7896318421699
epoch:  6  loss:  430.099285031436
epoch:  7  loss:  425.2490164996125
epoch:  8  loss:  421.2367142714211
epoch:  9  loss:  416.76439364926773
epoch:  10  loss:  413.712205515214
epoch:  11  loss:  410.12097829481354
epoch:  12  loss:  406.7480441683074
epoch:  13  loss:  403.4071175185745
epoch:  14  loss:  399.6728444245964
epoch:  15  loss:  395.5558434764698
epoch:  16  loss:  389.3890879877731
epoch:  17  loss:  386.92169112192096
epoch:  18  loss:  381.5432640342069
epoch:  19  loss:  374.8857741712077
epoch:  20  loss:  373.188395698711
epoch:  21  loss:  366.3218131561066
epoch:  22  loss:  362.66298918341823
epoch:  23  loss:  358.14281021215857
epoch:  24  loss:  354.2800417264648
epoch:  25  loss:  347.10938895953245
epoch:  26  loss:  345.1015045643212
epoch

In [11]:
model1.eval()

correct_count = 0
total = 0

with torch.no_grad():
    for i in range(280):
        input = X_test_tensor[i]
        label = y_test_tensor[i]
        params = model1(input)
        pred_result = generation(params).squeeze(-1)
        pred = (pred_result > 0.5).float()
        if pred == label:
            correct_count += 1
        total += 1
print(f'Correct predictions: {correct_count}')
print(f'Accuracy: {correct_count / total:.2f}')

Correct predictions: 187
Accuracy: 0.67


In [12]:
model2.eval()

correct_count = 0
total = 0

with torch.no_grad():
    for i in range(280):
        input = X_test_tensor[i]
        label = y_test_tensor[i]
        params = model2(input)
        pred_result = generation(params).squeeze(-1)
        pred = (pred_result > 0.5).float()
        if pred == label:
            correct_count += 1
        
        total += 1

print(f'Correct predictions: {correct_count}')
print(f'Accuracy: {correct_count / total:.2f}')

Correct predictions: 196
Accuracy: 0.70


In [13]:
model2.eval()

correct_count = 0
total = 0

with torch.no_grad():
    for i in range(280):
        input = X_val_tensor[i]
        label = y_val_tensor[i]
        params = model2(input)
        pred_result = generation(params).squeeze(-1)
        pred = (pred_result > 0.5).float()
        if pred == label:
            correct_count += 1
        total += 1

print(f'Correct predictions: {correct_count}')
print(f'Accuracy: {correct_count / total:.2f}')

Correct predictions: 187
Accuracy: 0.67
